In [1]:
import pandas as pd
import numpy as np

In [2]:
df_cards = pd.read_csv('input.txt', header=None, dtype='str', skiprows=1, names = ['card_row'])
df_numbers = pd.read_csv('input.txt', nrows=1, header = None, dtype='str')

In [3]:
numbers_called = df_numbers.to_numpy()[0]
numbers_called = numbers_called.astype(int)

In [4]:
df_cards[['1', '2', '3', '4', '5']] = df_cards['card_row'].str.split(expand = True)

In [5]:
array_of_card_ids = np.repeat(np.arange(100),5)

In [6]:
df_cards['card_id'] = array_of_card_ids
df_cards['row_id'] = [0,1,2,3,4] * 100

In [7]:
df_cards

,card_row,1,2,3,4,5,card_id,row_id
0,50 98 65 14 47,50,98,65,14,47,0,0
1,0 22 3 83 46,0,22,3,83,46,0,1
2,87 93 81 84 58,87,93,81,84,58,0,2
3,40 35 28 74 48,40,35,28,74,48,0,3
4,45 99 59 37 64,45,99,59,37,64,0,4
...,...,...,...,...,...,...,...,...
495,88 62 76 78 95,88,62,76,78,95,99,0
496,64 65 36 58 22,64,65,36,58,22,99,1
497,7 21 98 93 42,7,21,98,93,42,99,2
498,79 99 9 89 10,79,99,9,89,10,99,3


In [8]:
df_melted = pd.melt(df_cards, var_name = 'column_id', value_name = 'number_on_card', id_vars = ['card_id', 'row_id'], value_vars = ['1', '2', '3', '4', '5'])

In [9]:
for column in ['column_id', 'number_on_card']:
    df_melted[column] = df_melted[column].astype(int)
df_melted['column_id'] = df_melted['column_id'] - 1
df_melted['number_called'] = 0

In [10]:
df_melted

,card_id,row_id,column_id,number_on_card,number_called
0,0,0,0,50,0
1,0,1,0,0,0
2,0,2,0,87,0
3,0,3,0,40,0
4,0,4,0,45,0
...,...,...,...,...,...
2495,99,0,4,95,0
2496,99,1,4,22,0
2497,99,2,4,42,0
2498,99,3,4,10,0


In [11]:
# win condition
# group by card_id, if sum of "number_called" in a column or row is 5 then win

def check_if_winners(df):
    '''
    Checks if any winning bingo cards
    Returns card_id of winner if so
    Cards could be winning by rows or columns, on the same turn
    So we build up a list of winners
    '''

    winning_cards = []

    df_check_rows = df[['card_id', 'row_id', 'number_called']].groupby(['card_id', 'row_id']).sum().reset_index()
    df_check_cols = df[['card_id', 'column_id', 'number_called']].groupby(['card_id', 'column_id']).sum().reset_index()

    if df_check_rows['number_called'].max() == 5:
        winners_by_rows = df_check_rows[df_check_rows['number_called'] == 5]['card_id'].to_numpy()
        winning_cards.append(winners_by_rows)
    elif df_check_cols['number_called'].max() == 5:
        winners_by_cols = df_check_cols[df_check_cols['number_called'] == 5]['card_id'].to_numpy()
        winning_cards.append(winners_by_cols)

    return winning_cards

In [12]:
df_lets_play_bingo = df_melted.copy(deep = True)

for i, number in enumerate(numbers_called):
    print('Call ' + str(i) + ' calling '+ str(number))
    #df_melted[df_melted['number_on_card'] == number]['number_called'] = 1
    df_lets_play_bingo.loc[df_lets_play_bingo['number_on_card'] == number, 'number_called'] = 1
    win_check = check_if_winners(df_lets_play_bingo)
    if len(win_check) > 0:
        print('We have a winner! Card ID(s) ' + str(win_check))
        break

Call 0 calling 23
Call 1 calling 30
Call 2 calling 70
Call 3 calling 61
Call 4 calling 79
Call 5 calling 49
Call 6 calling 19
Call 7 calling 37
Call 8 calling 64
Call 9 calling 48
Call 10 calling 72
Call 11 calling 34
Call 12 calling 69
Call 13 calling 53
Call 14 calling 15
Call 15 calling 74
Call 16 calling 89
Call 17 calling 38
Call 18 calling 46
Call 19 calling 36
Call 20 calling 28
Call 21 calling 32
We have a winner! Card ID(s) [array([48], dtype=int64)]


In [13]:
def calculate_score(last_number_called, card_id):
    # get unmarked numbers
    df_unmarked = df_lets_play_bingo[(df_lets_play_bingo['card_id'] == card_id) & (df_lets_play_bingo['number_called'] == 0)]['number_on_card']
    sum_of_unmarked = df_unmarked.sum()
    final_score = sum_of_unmarked * last_number_called
    return final_score

In [14]:
# calculate the winning score
calculate_score(number, 48)

31424